In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from nltk.corpus import stopwords 
import sklearn.ensemble as ens
import sklearn.feature_extraction.text as fet
from bs4 import BeautifulSoup
import re

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("../input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip", header=0, delimiter="\t", quoting=3)
test = pd.read_csv("../input/word2vec-nlp-tutorial/testData.tsv.zip", header=0, delimiter="\t", quoting=3)

In [ ]:
train

In [ ]:
test

In [ ]:
def pre_process_reviews( review ):
    obj = BeautifulSoup(review).get_text()   
    process = re.sub("[^a-zA-Z]", " ", obj) 
    words = process.lower().split()                             
    stops = set(stopwords.words("english"))                  
    mean_words = [w for w in words if not w in stops]   
    return( " ".join( mean_words ))   

In [ ]:
train['review']=[pre_process_reviews(review) for review in train['review']]
clean_train_reviews = []

for i in range( 0, train["review"].size ): clean_train_reviews.append( pre_process_reviews( train["review"][i] ))

In [ ]:
vector = fet.CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 25000) 

train_data_features = vector.fit_transform(clean_train_reviews)
train_data_features = train_data_features.toarray()

In [ ]:
rfc = ens.RandomForestClassifier(n_estimators = 200) 
rfc = rfc.fit( train_data_features, train["sentiment"] )

In [ ]:
clean_test_reviews = []
for i in range(0,len(test["review"])):
    clean_review = pre_process_reviews(test["review"][i])
    clean_test_reviews.append(clean_review)

test_data_features = vector.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

result = rfc.predict(test_data_features)

In [ ]:
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

output.to_csv( "submission.csv", index=False, quoting=3 )